In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import torch
from torch import nn

In [5]:
from src.networks.core.tensor_shape import TensorShape
from src.networks.skip_nets.modulated_additive_skip_net import ModulatedAdditiveSkipNet
from src.networks.core.net import Net
from src.networks.skip_nets.modulated_dense_skip_net import ModulatedDenseSkipNet
from src.networks.core.parallel_net import ParallelNet
from src.module_analysis import count_parameters
from src.networks.skip_nets.dense_skip_net import DenseSkipNet
from src.networks.core.layer_connections import LayerConnections
from src.networks.skip_nets.additive_skip_net import AdditiveSkipNet

def create_mod_dense(from_idx: int, to_idx: int, num_f: int):
    if from_idx != to_idx:
        return Net.as_net(create_ones_linear(num_f, 2))
    else:
        return Net.as_net(nn.Identity())

def create_mod_additive(from_idx: int, to_idx: int, num_f: int):
    if from_idx != to_idx:
        return Net.as_net(create_ones_linear(num_f, num_f))
    else:
        return Net.as_net(nn.Identity())

def create_ones_linear(in_features, out_features):
    l = nn.Linear(in_features, out_features)
    l.weight.data[:] = 1.0
    l.bias.data[:] = 0.0
    return l

def create_layer(i, is_last, in_features, out_features):
    return nn.Sequential(
        create_ones_linear(in_features, out_features),
        nn.CELU() if not is_last else nn.Identity(),
    )

num_features=3
# net = AdditiveSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=4,
#     num_features=num_features,
#     layer_connections='full',
#     initial_direct_connection_weight=1.0,
#     initial_skip_connection_weight=0.01,
#     return_dense=False,
#     weights_trainable=False,
# )
# net = DenseSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=10,
#     num_features=num_features,
#     layer_connections='full'
# )
# net = ParallelNet.from_layer_provider(create_layer, in_size=num_features, out_sizes=[5, 7, 11, 13])
net = ModulatedDenseSkipNet.from_layer_provider(
    layer_provider=create_layer,
    modulator_provider=create_mod_dense,
    in_size=num_features,
    layer_out_sizes=[11, 23, 17, 13],
)
# net = ModulatedAdditiveSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     modulator_provider=create_mod_additive,
#     num_features=num_features,
#     num_layers=4,
#     # return_dense=True,
# )
print(net.count_parameters())
print(net)

out = net(torch.ones(111, 5, 19, num_features))
print(out.shape)
# print(out)
# print(net.forward_shape(TensorShape(features=num_features, s_0=111, s_1=5)))

1390
ModulatedDenseSkipNet(
  (layers): NetList(
    (0): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=3, out_features=11, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (1): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=13, out_features=23, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (2): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=27, out_features=17, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (3): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=23, out_features=13, bias=True)
        (1): Identity()
      )
    )
  )
  (connection_modulators[0][0]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[1][0]): TorchNet(
    (torch_module): Linear(in_features=3, out_features=2, bias=True)
  )
  (connection_modulators[1][1]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[2][0]): TorchNet

In [6]:
90 * 3 + 3 * 3 * 18 + 3 * 3 * 3 + 3 * 3 * 1

468

In [7]:
3 * (90 + 18 + 3 + 1) + 468

804

In [8]:
804

804

In [9]:
from src.networks.core.torch_wrappers.conv_net import Conv2dNet

conv = Conv2dNet(2, 3, 4, padding='same')
conv

Conv2dNet(2, 3, kernel_size=(4, 4), stride=(1, 1), padding=same)

In [10]:
from src.networks.core.torch_wrappers.linear_net import LinearNet

lin = LinearNet(2, 3)
lin(torch.ones(2))

tensor([ 0.4794, -0.7732, -0.1890], grad_fn=<ViewBackward0>)

In [11]:
from src.networks.multihead_self_attention import MultiheadSelfAttention

mhsa = MultiheadSelfAttention(10, 5)
mhsa.forward_shape(TensorShape(features=10, s_0=None))
mhsa(torch.ones(123, 9, 10))

tensor([[[ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         ...,
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041]],

        [[ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         ...,
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041]],

        [[ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0

In [12]:
from src.networks.core.layer_connections import *
from src.integer_sequences import *

(LayerConnectionsBuilder(12)
    .sequential()
    .every_step_receives(factorial).every_step_to_output().to_np())

array([[ 0,  0],
       [ 0, 12],
       [ 1,  1],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 1,  8],
       [ 1,  9],
       [ 1, 10],
       [ 1, 11],
       [ 1, 12],
       [ 2,  2],
       [ 2,  3],
       [ 2,  4],
       [ 2,  5],
       [ 2,  6],
       [ 2,  7],
       [ 2,  8],
       [ 2,  9],
       [ 2, 10],
       [ 2, 11],
       [ 2, 12],
       [ 3,  3],
       [ 3, 12],
       [ 4,  4],
       [ 4, 12],
       [ 5,  5],
       [ 5, 12],
       [ 6,  6],
       [ 6,  7],
       [ 6,  8],
       [ 6,  9],
       [ 6, 10],
       [ 6, 11],
       [ 6, 12],
       [ 7,  7],
       [ 7, 12],
       [ 8,  8],
       [ 8, 12],
       [ 9,  9],
       [ 9, 12],
       [10, 10],
       [10, 12],
       [11, 11],
       [11, 12],
       [12, 12]], dtype=int32)

In [1]:
from src.torch_device import set_default_torch_device
from torch.distributions import Normal
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch.distributions import Categorical
from torch import optim, nn
from src.reinforcement_learning.a2c.episodic.a2c import A2C
from src.networks.core.seq_net import SeqNet
import gymnasium as gym

device = set_default_torch_device("cuda:0")
print(f'using device {device}')

env = gym.make("BipedalWalker-v3")

def init_network(out_size: int, is_actor: bool):
    return SeqNet.from_layer_provider(
        layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.CELU() if not is_last_layer else (nn.Tanh() if is_actor else nn.Identity())
        ),
        in_size=24,
        out_sizes=[32, 32, 32, out_size]
    )

def select_action(logits: torch.Tensor):
    dist = Normal(loc=logits, scale=0.15)
    
    action = dist.sample()
    
    return action.detach().cpu().numpy(), dist.log_prob(action)

actor = init_network(4, True)
critic = init_network(1, False)

A2C(
    env=env,
    actor_network=actor,
    actor_network_optimizer=optim.Adam(actor.parameters()),
    critic_network=critic,
    critic_network_optimizer=optim.Adam(critic.parameters()),
    critic_loss=nn.MSELoss(),
    select_action=select_action,
    gamma=0.9,
    on_episode_done=lambda a2c, i_episode, is_best_episode, reward, end_timestep: print(f'{i_episode:>4}: {reward:> 8.1f}, {end_timestep:>6}'),
).find_optimal_policy(1000)

using device cuda:0
   0:    -94.8,     98
   1:    -95.9,     69
   2:    -96.2,     60
   3:    -97.0,     68
   4:    -97.9,     83
   5:    -96.4,     83
   6:    -94.1,    149
   7:    -95.0,     71
   8:   -117.8,     73
   9:   -117.2,     69
  10:    -95.2,     67
  11:    -96.5,     81
  12:    -95.6,     77
  13:   -119.5,     96
  14:    -95.7,     66
  15:    -95.8,     77
  16:    -97.1,     83
  17:    -96.1,     99
  18:    -97.4,     71
  19:    -94.9,     84
  20:    -95.2,     86
  21:    -95.2,     76
  22:    -96.2,     63
  23:    -95.8,     68
  24:    -96.3,     77
  25:   -119.4,    109
  26:    -94.1,     80
  27:    -95.6,     87
  28:    -93.7,    102
  29:    -97.2,     92
  30:    -94.6,     97
  31:    -96.4,     62
  32:    -97.0,     62
  33:    -93.7,    106
  34:   -120.8,    162
  35:    -94.2,    171
  36:    -95.5,    225
  37:    -95.0,     89
  38:    -94.7,     89
  39:    -93.9,     94
  40:    -94.6,     81
  41:    -95.3,     71
  42:    -93.6


KeyboardInterrupt



In [ ]:
nn.Conv2d